# 1. Kaggle Datasets

## 1.1. What is Kaggle?

## 1.2. Getting Started

### Sign up


### Create an API Token

![User settings menu](figures/kaggle_my_account.png)

Scroll down a bit and you will find an API section. Click '`Create New API Token`' button.

![Create New Kaggle API Token](figures/create_new_kaggle_api_token.png)


### Copy JSON file

In [1]:
# This is how you install a python library
!pip install kaggle

In [7]:
# type your Kaggle API token here
token = {"username":"bigdata","key":"a1b2c3d4"}

In [29]:
# This cell is only for advanced users. Run this cell and you can safely move on to the next cell
import os
from pathlib import Path
import json
import platform

# creates and places the token file at a desired location
home = str(Path.home())
kaggle_root = os.path.join(*[home, '.kaggle'])
os.mkdir(kaggle_root)
with open(os.path.join(*[kaggle_root, 'kaggle.json']), 'w') as file:
    json.dump(token, file)

# make the key file accessible only to the owner
if platform.system() == 'Windows':
    !attrib -R {os.path.join(*[kaggle_root, 'kaggle.json'])}
else:
    !chmod 600 {os.path.join(*[kaggle_root, 'kaggle.json'])}

In [31]:
# This is how you download datasets from Kaggle
# !kaggle datasets download -d <path-to-dataset> -p <download-location>
#    <path-to-dataset> is what comes after http://www.kaggle.com/ in the dataset URL.
#                      for example, the avocado dataset sits under https://www.kaggle.com/neuromusic/avocado-prices
#    <download-location> is where you want to download data.
!kaggle datasets download -d neuromusic/avocado-prices -p data

avocado-prices.zip: Skipping, found more recently modified local copy (use --force to force download)


In [32]:
# Unzipping files.
if platform.system() == 'Windows':
    this_file_path = !echo %cd%
    data_path = os.path.join(*[this_file_path[0], 'data'])
    print('[IMPORTANT] No automatic unzipping supported on Windows.')
    print('You have to open `File Explorer` and manually unzip `' + data_path + '\\avocado-prices.zip`')
    print('Make sure `avocado.csv` file in the zip file is placed directly under `data` folder:')
    print('|- ica02')
    print('    |- data')
    print('        |- avocado.csv')
    print('    |- How_to_Read_and_Represent_Data.ipynb')
else:
    !unzip ./data/new-york-city-current-job-postings.zip ./data

[IMPORTANT] No automatic unzipping supported on Windows.
You have to open `File Explorer` and manually unzip `E:\dev\bigdata\in-class-assignments\ica02\data\avocado-prices.zip`
Make sure `avocado.csv` file in the zip file is placed directly under `data` folder:
|- ica02
    |- data
        |- avocado.csv
    |- How_to_Read_and_Represent_Data.ipynb


In [36]:
import pandas as pd

DF = pd.read_csv('data/avocado.csv')
DF.head(5)

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [39]:
DF['Total Bags']

0           8696.87
1           9505.56
2           8145.35
3           5811.16
4           6183.95
5           6683.91
6           8318.86
7           6829.22
8          11388.36
9           8625.92
10          8205.66
11         10123.90
12          8756.75
13          6034.46
14          9267.36
15          9286.68
16          7990.10
17         10306.73
18         10880.36
19         10443.22
20          9225.89
21         11847.02
22         13192.69
23         11287.48
24         24431.90
25         29898.96
26         26662.08
27         21875.65
28         29002.59
29         22775.21
            ...    
18219     945638.02
18220     977084.84
18221     936859.49
18222     914409.26
18223    1005593.78
18224    1089861.24
18225     166747.85
18226     129353.55
18227     176465.63
18228     250090.37
18229     218560.51
18230     155725.83
18231     188559.45
18232     205409.91
18233     129911.47
18234     128267.76
18235     126261.89
18236     199330.12
18237      10806.44
